# Trajectory planning of self driving cars with a continuous method
I implement a simple version of the trajectory planning method described in [Trajectory Planning for BERTHA - a Local, Continuous Method](https://pdfs.semanticscholar.org/bdca/7fe83f8444bb4e75402a417053519758d36b.pdf). To make things easier, I only focus on the longitudinal control. This means that I only plan the velocity decomposition of the car. The mathematical basics behind the method are described below. 

The target of this method is minimizing the cost of a trajectory, given by a cost functional with respect to constraints and thus finding an optimal trajectory. 

# Cost-functional
The desired driving behaviour is described in the functional

$$ J(s(t)) = \frac{1}{2} \int_{t}^{t+T} w_{v}(v(t) - v_{des})^{2} + w_{a}a(t)^{2} + w_{j}j(t)^{2} dt. $$

$v(t)$, $a(t)$ and $j(t)$ describe the velocity, acceleration and jerk of the car. They are related through the planned path coordinate $s(t)$:

-
$$ v(t) = \dot{s}(t) = \frac{d s}{d t}, $$
-
$$ a(t) = \ddot{s}(t) =\frac{d v}{d t}, $$
-
$$ j(t) = \dddot{s}(t) = \frac{d a}{d t}. $$

$v_{des}$ is the desired velocity and $T$ is the optimization timespan. $w_{v}$, $w_{v}$ and $w_{v}$ are weights to manually emphasize the importance of each term in respect to the other terms inside the cost functional. 

# Constraints

## Physical constraints
A vehicle has a certain velocity and acceleration limit,

-
$$ v(t) \le v_{max}, $$
-
$$ a(t) \le a_{max}. $$

## Obstacle constraints
The only obstacles I focus on here are other vehicles, that intersect the self driving cars path. The figure below shows an example. The green vehicle is the self driving car. It's path $s(t)$ is intersected by the red cars path. This is why the red car is a dynamic obstacle. 
![](img/small.png)

The instant when the red car reaches the intersection is referred to as $t_{a}$. The moment when the red car has completely left behind the intersection is referred to as $t_{b}$ with $t_{b} \ge t_{a}$. This means that the timespan, in which the green vehicle is not allowed to drive trough the intersection is $t_{b} - t_{a}$. The width of the intersection is referred to as $2\Delta s = s_{b} - s_{a}$ with $s_{b} \ge s_{a}$.

In the diagram below the path-time diagram of the green car is depicted with an optimization timespan of $T=10 s$. In it the red vehicle is represented as a blue rectangle. The coordinates of the corners are: Bottom left $(t_{a}, s_{a})$, top left $(t_{a}, s_{b})$, bottom right $(t_{b}, s_{a})$ and top right $(t_{b}, s_{b})$. 

![](img/raumzeit.png)

Since the green vehicles trajectory is not supposed to go through the blue box the resulting obstacle constraint is
$$ | s(t) - \overline{s}| \ge \Delta s, \quad \in [t_{a}, t_{b}]. $$

$\bar{s}$ is equal to $\frac{1}{2} (s_{a} + s_{b})$. 

The obstacle constraint makes the optimization non-convex, which leads to some problems. One of them is that we obtain several (in this case two) local minima. This means that the solution of an optimization can vary with different initializations. In the left diagram below two different initializations are displayed (red and green line). The right diagram shows that the initializations converge differently.

![](img/initial.png)

To find out which of the trajectories is the global optimum, the costs of the trajectories, according to $J(s(t))$ have to be compared. 

Another way to find the local minima independently of the initialization is to split up the obstacle constraint into two linear constraints and afterwards performing an optimization for each. The linear obstacle constraints are 

-
$$ s(t) \ge s_{b}, \quad \in [t_{a}, t_{b}], $$
-
$$ s(t) \le s_{a}, \quad \in [t_{a}, t_{b}]. $$

The right diagram below shows the first linear constraint and the optimization result. The left diagram shows the second linear constraint and the according optimization result. 

![](img/konvex.png)

This method makes it possible to find all local minima and then find the global minimum by performing a cost comparison with respect to $J(s(t))$.